In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho
from pytenet.operation_thc import apply_thc_mpo_and_compress, add_mps_and_compress
from pytenet.operation import vdot, add_mps
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
import scipy.sparse.linalg as spla
from pytenet.krylov_time_evo import ED_time_evo, Krylov_evo_using_vecs_single_step, Krylov_time_evo_using_vecs, Krylov_evo_using_built_space, create_Krylov_space, gram_schmidt, Krylov_evo_using_built_mps_space

In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


In [3]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 58, 96, 96, 58, 16, 1]


In [4]:
#ref Hamiltonian as matrix
H_ref = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_thc = scipy.io.mmread('../data_water/H_water_thc.mtx').tocsr()

#initial state
filename = f"/work_fast/ge49cag/code_datas" + f"/water_ground_ionization.pkl"
with open(filename, 'rb') as file:
    initial_state = pickle.load(file)

#thc mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)
r_THC = int((len(H_mu_nu_list_spin_layer)-1)**(1/2) / 2)
    

In [ ]:
N_krylov = 4

# psi_krylov_ref = Krylov_time_evo_using_vecs(H_ref, N_krylov, initial_state.as_vector(), 1, 0.03)
# psi_ed = ED_time_evo(H_ref, initial_state.as_vector(), 0.03)
# print('total swift:', norm(psi_ed - initial_state.as_vector()))
# print(norm(psi_krylov_ref - psi_ed))

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/krylov_time_evo.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  TN[i, j] = np.vdot(Krylov_vector_list[i], H@Krylov_vector_list[j])


[[-8.19779895e-01+5.72583778e-01j]
 [ 6.25612246e-03+8.30175739e-03j]
 [ 5.19439204e-04-6.36076473e-04j]
 [-8.09283115e-05-4.25433979e-05j]]
total swift: 1.9077630331028186
5.4866900147348975e-06


In [6]:
'''
for i in range(n):
first update foldername
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)
then calculate time-evolved state (with re-ortho)
'''
#不需要创造很多文件夹储存文件，对每个文件好好命名然后都存在同一个文件夹即可

'\nfor i in range(n):\nfirst update foldername\ngenerate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername)\nthen calculate time-evolved state (with re-ortho)\n'

In [ ]:
# only use max_bond, don't set truncation tol!
# need introduce min_bond again to reduce error!
dt_list = [0.005, 0.008, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.3, 0.5]
#max_bond_list = [25, 30, 35, 40]
max_bond_list = [25]

for max_bond in max_bond_list:
    print (max_bond)
    space_test = create_Krylov_space(N_krylov, H_mu_nu_list_spin_layer, copy.deepcopy(initial_state), 0, max_bond, r_THC)
    for dt in dt_list:
        print (dt)
        psi_krylov_ref = Krylov_time_evo_using_vecs(H_ref, N_krylov, initial_state.as_vector(), 1, dt)
        psi_ed = ED_time_evo(H_ref, initial_state.as_vector(), dt)
        vec_mps_test = Krylov_evo_using_built_mps_space(mpo_ref, space_test, max_bond, dt)
        print('trunc error', norm(vec_mps_test.as_vector() - psi_krylov_ref))
        print('krylov error', norm(psi_krylov_ref - psi_ed))
        print('total error', norm(vec_mps_test.as_vector() - psi_ed))
        
# 用总时间1au画误差分析
# 用总时间1fs = 40au画D=30/40时候的误差图（或许可以说明误差累积/以及由于纠缠，误差增长变快）
# 并且比较内存使用
# 或许也可以画

25
0.005
[[ 9.12277076e-01+4.09569907e-01j]
 [ 7.02891426e-04-1.59174003e-03j]
 [-2.17667192e-05-8.57932323e-06j]
 [-1.45955943e-07+4.09603669e-07j]]
trunc error 0.0003381346092251197
krylov error 4.321174892779629e-09
total error 0.0003381346163018882
0.008
[[ 7.80595680e-01+6.25030106e-01j]
 [ 1.71833703e-03-2.18978036e-03j]
 [-4.93515830e-05-3.38181422e-05j]
 [-9.26340238e-07+1.51871980e-06j]]
trunc error 0.0005432200863476756
krylov error 2.8293402217984362e-08
total error 0.0005432201560540054
0.01
[[ 6.64499916e-01+7.47280237e-01j]
 [ 2.57103609e-03-2.34335723e-03j]
 [-6.82631881e-05-6.37196399e-05j]
 [-2.19481749e-06+2.68852285e-06j]]
trunc error 0.0006783577907883388
krylov error 6.901748649253585e-08
total error 0.0006783580188202916
0.02
[[-1.16866495e-01+9.93123267e-01j]
 [ 6.91712506e-03+6.43572598e-04j]
 [-2.52202463e-05-3.69422311e-04j]
 [-2.69548007e-05+5.50050066e-06j]]
trunc error 0.001384447772776346
krylov error 1.0965504568938947e-06
total error 0.001384456308420778

In [ ]:
# dt = 0.03

# Krylov_mps_list = copy.deepcopy(space_test)
# TN = np.zeros([len(Krylov_mps_list),len(Krylov_mps_list)])
# for i in range (TN.shape[0]):
#     for j in range (TN.shape[1]):
#         if abs(i - j) < 2:
#             TN[i, j] = ptn.operation.operator_inner_product(Krylov_mps_list[i], mpo_ref, Krylov_mps_list[j])
            
# c1 = np.zeros([len(Krylov_mps_list), 1])
# c1[0,0] = 1
# exp_TN = spla.expm(-1j*dt*TN)
# c_reduced = exp_TN@ c1

# psi_evloved = copy.deepcopy(Krylov_mps_list[0])
# psi_evloved.A[0] = c_reduced[0] *psi_evloved.A[0]

# for i in range (1, len(Krylov_mps_list), 1):
#     temp = copy.deepcopy(Krylov_mps_list[i])
#     temp.A[0] = c_reduced[i] *temp.A[0]
#     psi_evloved = add_mps(psi_evloved, temp)
    
# print(norm(psi_evloved.as_vector() - psi_krylov_ref))
# psi_evloved.orthonormalize('right')
# psi_evloved.orthonormalize('left')


# psi_evloved.compress_direct_svd_right_max_bond(0, max_bond)
# psi_evloved.orthonormalize('right')
# psi_evloved.orthonormalize('left')
    
# print(norm(psi_evloved.as_vector() - psi_krylov_ref))
# print(norm(psi_evloved.as_vector() - psi_ed))
# print(norm(psi_krylov_ref - psi_ed))

In [ ]:
# /tmp/ipykernel_3032840/2463284093.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
#   TN[i, j] = ptn.operation.operator_inner_product(Krylov_mps_list[i], mpo_ref, Krylov_mps_list[j])
# 3.5432687797238603e-06
# 9.462310845684574e-06
# 1.093342968130352e-05
# 5.4866900147348975e-06

9.462310845684574e-06
1.093342968130352e-05
5.4866900147348975e-06


In [ ]:
# space_test_vec = []
# for i in range (len(space_test)):
#     temp = space_test[i].as_vector()
#     temp /= norm(temp)
#     space_test_vec.append(temp)

# space_test_vec = gram_schmidt(space_test_vec)

space_test_vec = [mps_krylov.as_vector() for mps_krylov in space_test]
space_test_vec = gram_schmidt(space_test_vec)

In [ ]:
# #only use max_bond, don't set truncation tol!
# vec_test = Krylov_evo_using_built_space(H_ref, space_test_vec, 0.05)
# print(norm(vec_test - psi_krylov_ref))
# print(norm(vec_test - psi_ed))

1.1907312675090874e-12
4.078012058453979e-05


In [ ]:
# dt = 0.03

# D = 45:



# D = 40:
# 7.159356839258043e-13 (trunc error)
# 5.486690011281797e-06 (total error)

# D = 35
# 3.537471270532817e-06 (trunc error)
# 6.522437168647089e-06 (total error)

# D= 30
# 0.0004761841209985205 (trunc error)
# 0.00047618669594564364 (total error)